In [ ]:
import sympy as sym
import numpy as np
import scipy as sp
from scipy import linalg,spatial

import matplotlib.pyplot as plt

In [ ]:
# podatki
ρ = 2700
E = 70e9
L = 1

h = 0.030
A = h*h # mm^2
I = h*h**3/12 # mm^4

In [ ]:
# transformacijska matrika
def T(α):
    
    T = np.array([[ np.cos(α), np.sin(α), 0,         0,         0, 0],
                   [-np.sin(α), np.cos(α), 0,         0,         0, 0],
                   [         0,         0, 1,         0,         0, 0],
                   [         0,         0, 0, np.cos(α), np.sin(α), 0],
                   [         0,         0, 0,-np.sin(α), np.cos(α), 0],
                   [         0,         0, 0,         0,         0, 1]])
        
    return T

In [ ]:
# masna matrika
def M_e(A, L, ρ, α):

    M_lok = ρ*A*L*np.array([[1/3,         0,         0, 1/6,         0,         0],
                            [  0,     13/35,  11*L/210,   0,      9/70, -13*L/420],
                            [  0,  11*L/210,  L**2/105,   0,  13*L/420, -L**2/140],
                            [1/6,         0,         0, 1/3,         0,         0],
                            [  0,      9/70,  13*L/420,   0,     13/35, -11*L/210],
                            [  0, -13*L/420, -L**2/140,   0, -11*L/210,  L**2/105]])
    
    return T(α).T @ M_lok @ T(α)

In [ ]:
# togostna matrika
def K_e(A, E, I, L, α):

    K_lok = E/L*np.array([[ A,          0,      0, -A,          0,      0],
                          [ 0,  12*I/L**2,  6*I/L,  0, -12*I/L**2,  6*I/L],
                          [ 0,      6*I/L,    4*I,  0,     -6*I/L,    2*I],
                          [-A,          0,      0,  A,          0,      0],
                          [ 0, -12*I/L**2, -6*I/L,  0,  12*I/L**2, -6*I/L],
                          [ 0,      6*I/L,    2*I,  0,     -6*I/L,    4*I]])
    
    return T(α).T @ K_lok @ T(α)

In [ ]:
# vozlišča in elementi
%matplotlib inline
a = 1.2*L
b = 0.5*L
vozlisca = np.array([[0,0],[0,b],[a,b],[a,0]])
elementi = np.array([[0,1],[1,2],[2,3]])

[plt.plot(vozlisca[[i,j],0],vozlisca[[i,j],1],'-',c='C0') for i,j in elementi]
plt.plot(vozlisca[:,0],vozlisca[:,1],'o');
plt.axis('equal');

In [ ]:
# zgostitev mreže

In [ ]:
# sestav globalne masne in togostne matrike
M_glob = np.zeros((vozlisca.shape[0]*3,vozlisca.shape[0]*3))
K_glob = np.zeros((vozlisca.shape[0]*3,vozlisca.shape[0]*3))

for element in elementi:
    # dolžina elementa
    Le = sp.spatial.distance.euclidean(vozlisca[element[0]], vozlisca[element[1]])
    #Le = np.linalg.norm(vozlisca[element])
    
    # kot zasuka
    αe = np.arctan2(np.diff(vozlisca[element,1]) , np.diff(vozlisca[element,0]))[0]
    
    # indeksi prostostnih stopenj
    ind = np.array([3*element[0],3*element[0]+1,3*element[0]+2,3*element[1],3*element[1]+1,3*element[1]+2])
    
    # priševanje prispevkov posameznih elementov
    M_glob[ind[:,None],ind] += M_e(A, Le, ρ, αe)
    K_glob[ind[:,None],ind] += K_e(A, E, I, Le, αe)

In [ ]:
# ROBNI POGOJI
ind_vpetih_ps = np.array([0,1,2,-2])

# tvorimo matriko C

L = sp.linalg.null_space(C)

M_glob_rp = L.T @ M_glob @ L
K_glob_rp = L.T @ K_glob @ L

In [ ]:
# lastne vrednosti in lastni vektorji
eig_val, eig_vec = sp.linalg.eig(K_glob_rp, M_glob_rp)

# urejanje po velikosti
_ind = np.argsort(np.abs(eig_val))
eig_val = eig_val[_ind]
eig_vec = eig_vec[:,_ind]
eig_vec = L @ eig_vec

eig_freq = np.abs(eig_val)**0.5 / 2 / np.pi

In [ ]:
# lastne frekvence [Hz]
eig_freq[:4].round(1)

### Izris

In [ ]:
m = 0
s = 100

pomiki_x = eig_vec[0::3,m]
pomiki_y = eig_vec[1::3,m]

def_vozlisca = np.copy(vozlisca)
def_vozlisca[:,0] += pomiki_x *s
def_vozlisca[:,1] += pomiki_y * s

In [ ]:
%matplotlib inline
# nedeformirana geometrija
[plt.plot(vozlisca[[i,j],0],vozlisca[[i,j],1],'-',c='C0') for i,j in elementi]
plt.plot(vozlisca[:,0],vozlisca[:,1],'o');
# deformirana geometrija
[plt.plot(def_vozlisca[[i,j],0],def_vozlisca[[i,j],1],'-',c='C1') for i,j in elementi]
plt.plot(def_vozlisca[:,0],def_vozlisca[:,1],'o');
plt.axis('equal');

In [ ]:
# animacija lastne oblike